MODEL 3 (LLETRES I DÍGITS):

In [ ]:
#THIS ONE DOESN'T WORK PROPERLY

import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model  # type: ignore


# Carrega del model (ha de tenir 36 classes: A–Z + 0–9)
model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model3_cnn.keras")


class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconeixedor de Lletres i Nombres")
        self.root.geometry("600x700")

        # Estat
        self.dark_mode = False
        self.fullscreen = False

        self.root.bind("<F11>", self.toggle_fullscreen)
        self.root.bind("<Escape>", self.exit_fullscreen)

        # Configura tema inicial
        self.set_theme()

        # Títol
        self.header = tk.Label(
            self.root, text="Reconeixedor de Lletres i Nombres amb IA",
            font=("Segoe UI", 18, "bold"), bg=self.bg, fg=self.accent
        )
        self.header.pack(pady=(20, 10))

        # Canvas
        self.canvas_frame = tk.Frame(self.root, bg=self.card_bg, highlightbackground=self.border, highlightthickness=1)
        self.canvas_frame.pack(pady=15, ipadx=10, ipady=10)

        self.canvas = tk.Canvas(
            self.canvas_frame, width=300, height=300, bg=self.canvas_bg,
            highlightthickness=2, highlightbackground=self.border
        )
        self.canvas.pack(expand=True)

        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # Botons
        self.btn_frame = tk.Frame(self.root, bg=self.bg)
        self.btn_frame.pack(pady=10)

        self.btn_predict = ttk.Button(self.btn_frame, text="Predir", command=self.predict)
        self.btn_clear = ttk.Button(self.btn_frame, text="Esborrar", command=self.clear)
        self.btn_theme = ttk.Button(self.btn_frame, text="Mode Fosc", command=self.toggle_theme)

        self.btn_predict.grid(row=0, column=0, padx=10)
        self.btn_clear.grid(row=0, column=1, padx=10)
        self.btn_theme.grid(row=0, column=2, padx=10)

        # Resultat
        self.result_label = tk.Label(
            self.root,
            text="Dibuixa una lletra o nombre i prem «Predir».",
            font=("Segoe UI", 14, "bold"),
            bg=self.bg, fg=self.text
        )
        self.result_label.pack(pady=10)

        # Contenidor prediccions
        self.predictions_frame = tk.Frame(self.root, bg=self.bg)
        self.predictions_frame.pack_forget()
        self.pred_bars = []

        # Binding ratolí
        self.canvas.bind("<B1-Motion>", self.paint)

    # --- Gestió de tema ---
    def set_theme(self):
        if self.dark_mode:
            self.bg = "#2b1e17"
            self.card_bg = "#3b2b23"
            self.canvas_bg = "#f2e9e1"
            self.text = "#f5ede3"
            self.accent = "#d8a657"
            self.border = "#5a4539"
            self.bar_color = "#d8a657"
        else:
            self.bg = "#f4f1ed"
            self.card_bg = "#ffffff"
            self.canvas_bg = "#ffffff"
            self.text = "#3f3a36"
            self.accent = "#b45309"
            self.border = "#d6d3ce"
            self.bar_color = "#b45309"

        self.root.configure(bg=self.bg)
        style = ttk.Style()
        style.theme_use("clam")
        style.configure(
            "TButton",
            font=("Segoe UI", 11, "bold"),
            padding=6,
            background=self.accent,
            foreground="white",
            borderwidth=0,
        )
        style.map("TButton", background=[("active", self.bar_color)])
        style.configure(
            "TProgressbar",
            troughcolor=self.card_bg,
            background=self.bar_color,
            bordercolor=self.bar_color,
        )

    def toggle_theme(self):
        self.dark_mode = not self.dark_mode
        self.set_theme()

        self.root.configure(bg=self.bg)
        self.header.configure(bg=self.bg, fg=self.accent)
        self.result_label.configure(bg=self.bg, fg=self.text)
        self.canvas.configure(bg=self.canvas_bg, highlightbackground=self.border)
        self.canvas_frame.configure(bg=self.card_bg, highlightbackground=self.border)
        self.predictions_frame.configure(bg=self.bg)
        self.btn_frame.configure(bg=self.bg)
        for frame in self.pred_bars:
            frame.configure(bg=self.bg)
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Label):
                    widget.configure(bg=self.bg, fg=self.text)

    # --- Funcions bàsiques ---
    def toggle_fullscreen(self, event=None):
        self.fullscreen = not self.fullscreen
        self.root.attributes("-fullscreen", self.fullscreen)

    def exit_fullscreen(self, event=None):
        self.fullscreen = False
        self.root.attributes("-fullscreen", False)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="Dibuixa una lletra o nombre i prem «Predir».", fg=self.text)
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()
        self.predictions_frame.pack_forget()

    # --- Predicció ---
    def preprocess(self):
        img_resized = self.image.resize((28, 28))
        img_array = np.array(img_resized)

        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        _, img_array = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        img_ready = np.expand_dims(img_array, axis=(0, -1))
        return img_ready

    def idx_to_char(self, idx: int) -> str:
        """Converteix l'índex a lletra (A–Z) o número (0–9)."""
        if 0 <= idx < 26:
            return chr(idx + 65)  # A-Z
        elif 26 <= idx < 36:
            return str(idx - 26)  # 0-9
        else:
            return "?"

    def show_predictions(self, pred):
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()

        self.predictions_frame.pack(pady=10)

        sorted_indices = np.argsort(pred[0])[::-1]
        for idx in sorted_indices[:5]:
            percent = float(pred[0][idx]) * 100
            char = self.idx_to_char(idx)

            frame = tk.Frame(self.predictions_frame, bg=self.bg)
            frame.pack(fill="x", padx=60, pady=2)

            lbl = tk.Label(frame, text=f"{char}: {percent:.1f}%", width=10,
                           bg=self.bg, fg=self.text, anchor="w", font=("Segoe UI", 11, "bold"))
            lbl.pack(side="left")

            bar = ttk.Progressbar(frame, length=300, value=percent, maximum=100)
            bar.pack(side="left", padx=8)
            self.pred_bars.append(frame)

    def predict(self):
        img_ready = self.preprocess()
        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))
        predicted_char = self.idx_to_char(pred_class)

        self.result_label.config(text=f"Predicció: {predicted_char}", fg=self.accent)
        self.show_predictions(pred)


# --- Executar ---
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


MODEL 4 (NOMÉS LLETRES):

In [1]:
import tkinter as tk
from tkinter import *
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model # type: ignore

#Carrego el model entrenat

model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model4_cnn.keras")

# Interfície Tkinter

class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconèixer Lletres")

        # Canvas per dibuixar
        self.canvas = Canvas(root, width=280, height=280, bg="white")
        self.canvas.grid(row=0, column=0, columnspan=4)

        # Inicialitzar imatge PIL per capturar el dibuix
        self.image = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # Botons
        Button(root, text="Predir", command=self.predict).grid(row=1, column=0, sticky="ew")
        Button(root, text="Esborrar", command=self.clear).grid(row=1, column=1, sticky="ew")

        # Resultat
        self.label_result = Label(root, text="Dibuixa una lletra!", font=("Arial", 18))
        self.label_result.grid(row=2, column=0, columnspan=4)

        # Binding del ratolí
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(text="Dibuixa una lletra!")

    def preprocess(self):
        # Redimensionar a 28x28 amb Pillow modern
        img_resized = self.image.resize((28, 28), Image.Resampling.LANCZOS)

        # Convertir a array
        img_array = np.array(img_resized)

        # Invertir colors si fons és clar
        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        # Binarització
        _, img_array = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Convertir a int (0–255)
        img_array = img_array.astype("int")

        # Donar forma (1,28,28,1)
        img_ready = np.expand_dims(img_array, axis=-1)  # (28,28,1)
        img_ready = np.expand_dims(img_ready, axis=0)   # (1,28,28,1)

        return img_ready



    def predict(self):
        img_ready = self.preprocess()

        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))

        # Si tens només 26 classes (A-Z)
        try:
            predicted_letter = chr(pred_class + 65)
        except:
            predicted_letter = str(pred_class)

        self.label_result.config(text=f"Lletra predita: {predicted_letter}")

# Executar aplicació

if __name__ == "__main__":
    root = Tk()
    app = DrawApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


In [2]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model # type: ignore

# ==========================
# Carreguem el model entrenat
# ==========================
model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model4_cnn.keras")

# ==========================
# Interfície Tkinter
# ==========================
class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("✍️ Reconèixer Lletres")
        self.root.geometry("400x450")
        self.root.configure(bg="#f4f4f9")

        # Estils ttk
        style = ttk.Style()
        style.configure("TButton", font=("Helvetica", 12), padding=8)
        style.configure("TLabel", background="#f4f4f9", font=("Helvetica", 14))

        # Frame superior pel canvas
        frame_canvas = tk.Frame(self.root, bg="#f4f4f9")
        frame_canvas.pack(pady=15)

        self.canvas = tk.Canvas(frame_canvas, width=280, height=280, bg="white", bd=2, relief="ridge")
        self.canvas.pack()

        # Inicialitzar imatge PIL per capturar el dibuix
        self.image = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # Frame per botons
        frame_buttons = tk.Frame(self.root, bg="#f4f4f9")
        frame_buttons.pack(pady=10)

        self.btn_predict = ttk.Button(frame_buttons, text="🔍 Predir", command=self.predict)
        self.btn_predict.grid(row=0, column=0, padx=10)

        self.btn_clear = ttk.Button(frame_buttons, text="🧹 Esborrar", command=self.clear)
        self.btn_clear.grid(row=0, column=1, padx=10)

        # Resultat
        self.label_result = ttk.Label(self.root, text="👉 Dibuixa una lletra!", font=("Helvetica", 16, "bold"))
        self.label_result.pack(pady=20)

        # Binding del ratolí
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(text="👉 Dibuixa una lletra!")

    def preprocess(self):
        # Redimensionar a 28x28 amb Pillow modern
        img_resized = self.image.resize((28, 28), Image.Resampling.LANCZOS)

        # Convertir a array
        img_array = np.array(img_resized)

        # Invertir colors si fons és clar
        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        # Binarització
        _, img_array = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Convertir a int (0–255)
        img_array = img_array.astype("int")

        # Donar forma (1,28,28,1)
        img_ready = np.expand_dims(img_array, axis=-1)  # (28,28,1)
        img_ready = np.expand_dims(img_ready, axis=0)   # (1,28,28,1)

        return img_ready

    def predict(self):
        img_ready = self.preprocess()
        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))

        # Si tens només 26 classes (A-Z)
        try:
            predicted_letter = chr(pred_class + 65)
        except:
            predicted_letter = str(pred_class)

        self.label_result.config(text=f"✅ Lletra predita: {predicted_letter}")

# ==========================
# Executar aplicació
# ==========================
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [ ]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model # type:ignore

# ==========================
# Carreguem el model entrenat
# ==========================
model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model4_cnn.keras")

# ==========================
# Interfície Tkinter millorada
# ==========================
class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconèixer Lletres")
        self.root.geometry("420x500")
        self.root.resizable(False, False)
        self.root.configure(bg="#e9eef3")

        # --------------------------
        # Estils generals ttk
        # --------------------------
        style = ttk.Style()
        style.theme_use("clam")
        style.configure(
            "TButton",
            font=("Segoe UI", 11, "bold"),
            padding=8,
            relief="flat",
            background="#3b82f6",
            foreground="white"
        )
        style.map(
            "TButton",
            background=[("active", "#2563eb")],
            foreground=[("disabled", "#cccccc")]
        )

        style.configure("TLabel", background="#e9eef3", font=("Segoe UI", 13))

        # --------------------------
        # Frame superior pel canvas
        # --------------------------
        frame_canvas = tk.Frame(self.root, bg="#e9eef3")
        frame_canvas.pack(pady=20)

        self.canvas = tk.Canvas(
            frame_canvas,
            width=300,
            height=300,
            bg="white",
            bd=0,
            highlightthickness=2,
            highlightbackground="#cbd5e1"
        )
        self.canvas.pack()

        # Inicialitzar imatge PIL
        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # --------------------------
        # Botons
        # --------------------------
        frame_buttons = tk.Frame(self.root, bg="#e9eef3")
        frame_buttons.pack(pady=15)

        self.btn_predict = ttk.Button(frame_buttons, text="Predir", command=self.predict)
        self.btn_predict.grid(row=0, column=0, padx=15)

        self.btn_clear = ttk.Button(frame_buttons, text="Esborrar", command=self.clear)
        self.btn_clear.grid(row=0, column=1, padx=15)

        # --------------------------
        # Resultat estil targeta
        # --------------------------
        self.frame_result = tk.Frame(self.root, bg="#f1f5f9", bd=1, relief="solid")
        self.frame_result.pack(pady=15, ipadx=10, ipady=10, fill="x", padx=40)

        self.label_result = tk.Label(
            self.frame_result,
            text="Dibuixa una lletra al quadre superior.",
            bg="#f1f5f9",
            fg="#1e293b",
            font=("Segoe UI", 14, "bold"),
            wraplength=360,
            justify="center"
        )
        self.label_result.pack(pady=5)

        # --------------------------
        # Binding del ratolí
        # --------------------------
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(text="Dibuixa una lletra al quadre superior.", fg="#1e293b")

    def preprocess(self):
        img_resized = self.image.resize((28, 28), Image.Resampling.LANCZOS)
        img_array = np.array(img_resized)

        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        _, img_array = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        img_array = img_array.astype("int")

        img_ready = np.expand_dims(img_array, axis=-1)
        img_ready = np.expand_dims(img_ready, axis=0)
        return img_ready

    def predict(self):
        img_ready = self.preprocess()
        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))

        try:
            predicted_letter = chr(pred_class + 65)
        except:
            predicted_letter = str(pred_class)

        self.label_result.config(
            text=f"Lletra predita: {predicted_letter}",
            fg="#2563eb"
        )

# ==========================
# Executar aplicació
# ==========================
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [ ]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model #type: ignore

# ==========================
# Carreguem el model entrenat
# ==========================
model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model4_cnn.keras")

# ==========================
# Interfície millorada i amb tons càlids
# ==========================
class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconèixer Lletres")
        self.root.geometry("600x700")
        self.root.configure(bg="#f4f1ed")  # Fons beix clar

        # Estat pantalla completa
        self.fullscreen = False
        self.root.bind("<F11>", self.toggle_fullscreen)
        self.root.bind("<Escape>", self.exit_fullscreen)

        # --------------------------
        # Estil general
        # --------------------------
        style = ttk.Style()
        style.theme_use("clam")

        accent = "#b45309"         # Marró daurat per accents
        accent_active = "#92400e"  # Més fosc
        bg_frame = "#f4f1ed"
        card_bg = "#fdfcfb"

        style.configure(
            "TButton",
            font=("Segoe UI", 11, "bold"),
            padding=8,
            relief="flat",
            background=accent,
            foreground="white",
        )
        style.map(
            "TButton",
            background=[("active", accent_active)],
            foreground=[("disabled", "#cccccc")],
        )

        style.configure("TLabel", background=bg_frame, font=("Segoe UI", 13))
        style.configure(
            "warm.Horizontal.TProgressbar",
            troughcolor="#ede9e4",
            background=accent,
        )

        # --------------------------
        # Canvas per dibuixar
        # --------------------------
        frame_canvas = tk.Frame(self.root, bg=bg_frame)
        frame_canvas.pack(pady=20)

        self.canvas = tk.Canvas(
            frame_canvas,
            width=300,
            height=300,
            bg="white",
            bd=0,
            highlightthickness=2,
            highlightbackground="#d6d3ce",
        )
        self.canvas.pack()

        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # --------------------------
        # Botons
        # --------------------------
        frame_buttons = tk.Frame(self.root, bg=bg_frame)
        frame_buttons.pack(pady=15)

        self.btn_predict = ttk.Button(frame_buttons, text="Predir", command=self.predict)
        self.btn_predict.grid(row=0, column=0, padx=15)

        self.btn_clear = ttk.Button(frame_buttons, text="Esborrar", command=self.clear)
        self.btn_clear.grid(row=0, column=1, padx=15)

        # --------------------------
        # Resultat principal
        # --------------------------
        self.frame_result = tk.Frame(self.root, bg=card_bg, bd=1, relief="solid")
        self.frame_result.pack(pady=10, ipadx=10, ipady=10, fill="x", padx=40)

        self.label_result = tk.Label(
            self.frame_result,
            text="Dibuixa una lletra al quadre superior.",
            bg=card_bg,
            fg="#3f3a36",
            font=("Segoe UI", 16, "bold"),
            wraplength=480,
            justify="center",
        )
        self.label_result.pack(pady=5)

        # --------------------------
        # Contenidor de prediccions (ocult fins predicció)
        # --------------------------
        self.predictions_container = tk.Frame(self.root, bg=bg_frame)
        self.predictions_container.pack_forget()  # No mostrar inicialment

        canvas_scroll = tk.Canvas(self.predictions_container, bg=bg_frame, highlightthickness=0)
        scrollbar = ttk.Scrollbar(self.predictions_container, orient="vertical", command=canvas_scroll.yview)
        self.scrollable_frame = tk.Frame(canvas_scroll, bg=bg_frame)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas_scroll.configure(scrollregion=canvas_scroll.bbox("all")),
        )

        canvas_scroll.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas_scroll.configure(yscrollcommand=scrollbar.set)

        canvas_scroll.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        self.pred_bars = []

        # --------------------------
        # Binding del ratolí
        # --------------------------
        self.canvas.bind("<B1-Motion>", self.paint)

    # ==========================
    # Funcions bàsiques
    # ==========================
    def toggle_fullscreen(self, event=None):
        self.fullscreen = not self.fullscreen
        self.root.attributes("-fullscreen", self.fullscreen)

    def exit_fullscreen(self, event=None):
        self.fullscreen = False
        self.root.attributes("-fullscreen", False)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(
            text="Dibuixa una lletra al quadre superior.",
            fg="#3f3a36"
        )
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()
        self.predictions_container.pack_forget()

    # ==========================
    # Processament i predicció
    # ==========================
    def preprocess(self):
        img_resized = self.image.resize((28, 28), Image.Resampling.LANCZOS)
        img_array = np.array(img_resized)

        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        _, img_array = cv2.threshold(
            img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
        )
        img_array = img_array.astype("int")

        img_ready = np.expand_dims(img_array, axis=-1)
        img_ready = np.expand_dims(img_ready, axis=0)
        return img_ready

    def show_predictions(self, pred):
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()

        # Mostrar el contenidor només quan hi hagi prediccions
        self.predictions_container.pack(fill="both", expand=True, padx=40, pady=10)

        sorted_indices = np.argsort(pred[0])[::-1]

        for idx in sorted_indices:
            percent = float(pred[0][idx]) * 100
            letter = chr(idx + 65)
            frame = tk.Frame(self.scrollable_frame, bg="#f4f1ed")
            frame.pack(fill="x", padx=20, pady=3)

            lbl = tk.Label(
                frame,
                text=f"{letter}: {percent:.2f}%",
                font=("Segoe UI", 11, "bold"),
                bg="#f4f1ed",
                fg="#3f3a36",
                width=10,
                anchor="w",
            )
            lbl.pack(side="left")

            bar = ttk.Progressbar(
                frame,
                length=300,
                value=percent,
                maximum=100,
                style="warm.Horizontal.TProgressbar",
            )
            bar.pack(side="left", padx=10, fill="x", expand=True)

            self.pred_bars.append(frame)

    def predict(self):
        img_ready = self.preprocess()
        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))
        predicted_letter = chr(pred_class + 65)

        self.label_result.config(
            text=f"Lletra predita: {predicted_letter}",
            fg="#b45309"
        )
        self.show_predictions(pred)


# ==========================
# Executar aplicació
# ==========================
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()

In [4]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model # type: ignore


# Carrega del model
model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model4_cnn.keras")


class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconeixedor de Lletres")
        self.root.geometry("600x700")

        # Estat
        self.dark_mode = False
        self.fullscreen = False

        self.root.bind("<F11>", self.toggle_fullscreen)
        self.root.bind("<Escape>", self.exit_fullscreen)

        # Configura tema inicial
        self.set_theme()

        # Títol
        self.header = tk.Label(
            self.root, text="Reconeixedor de Lletres amb IA",
            font=("Segoe UI", 18, "bold"), bg=self.bg, fg=self.accent
        )
        self.header.pack(pady=(20, 10))

        # Canvas
        self.canvas_frame = tk.Frame(self.root, bg=self.card_bg, highlightbackground=self.border, highlightthickness=1)
        self.canvas_frame.pack(pady=15, ipadx=10, ipady=10)

        self.canvas = tk.Canvas(
            self.canvas_frame, width=300, height=300, bg=self.canvas_bg,
            highlightthickness=2, highlightbackground=self.border
        )
        self.canvas.pack(expand=True)  # <── Afegeix expand=True aquí

        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # Botons
        self.btn_frame = tk.Frame(self.root, bg=self.bg)
        self.btn_frame.pack(pady=10)

        self.btn_predict = ttk.Button(self.btn_frame, text="Predir", command=self.predict)
        self.btn_clear = ttk.Button(self.btn_frame, text="Esborrar", command=self.clear)
        self.btn_theme = ttk.Button(self.btn_frame, text="Mode Fosc", command=self.toggle_theme)

        self.btn_predict.grid(row=0, column=0, padx=10)
        self.btn_clear.grid(row=0, column=1, padx=10)
        self.btn_theme.grid(row=0, column=2, padx=10)

        # Resultat
        self.result_label = tk.Label(
            self.root,
            text="Dibuixa una lletra i prem «Predir».",
            font=("Segoe UI", 14, "bold"),
            bg=self.bg, fg=self.text
        )
        self.result_label.pack(pady=10)

        # Contenidor prediccions
        self.predictions_frame = tk.Frame(self.root, bg=self.bg)
        self.predictions_frame.pack_forget()
        self.pred_bars = []

        # Binding ratolí
        self.canvas.bind("<B1-Motion>", self.paint)

    # Gestió de tema
    def set_theme(self):
        if self.dark_mode:
            self.bg = "#2b1e17"         
            self.card_bg = "#3b2b23"    
            self.canvas_bg = "#f2e9e1"  
            self.text = "#f5ede3"       
            self.accent = "#d8a657"     
            self.border = "#5a4539"
            self.bar_color = "#d8a657"
        else:
            self.bg = "#f4f1ed"
            self.card_bg = "#ffffff"
            self.canvas_bg = "#ffffff"
            self.text = "#3f3a36"
            self.accent = "#b45309"
            self.border = "#d6d3ce"
            self.bar_color = "#b45309"

        self.root.configure(bg=self.bg)

        style = ttk.Style()
        style.theme_use("clam")
        style.configure(
            "TButton",
            font=("Segoe UI", 11, "bold"),
            padding=6,
            background=self.accent,
            foreground="white",
            borderwidth=0,
        )
        style.map("TButton", background=[("active", self.bar_color)])
        style.configure(
            "TProgressbar",
            troughcolor=self.card_bg,
            background=self.bar_color,
            bordercolor=self.bar_color,
        )

    def toggle_theme(self):
        self.dark_mode = not self.dark_mode
        self.set_theme()

        # Actualitza colors existents
        self.root.configure(bg=self.bg)
        self.header.configure(bg=self.bg, fg=self.accent)
        self.result_label.configure(bg=self.bg, fg=self.text)
        self.canvas.configure(bg=self.canvas_bg, highlightbackground=self.border)
        self.canvas_frame.configure(bg=self.card_bg, highlightbackground=self.border)
        self.predictions_frame.configure(bg=self.bg)
        self.btn_frame.configure(bg=self.bg)
        for frame in self.pred_bars:
            frame.configure(bg=self.bg)
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Label):
                    widget.configure(bg=self.bg, fg=self.text)

    # Funcions bàsiques
    def toggle_fullscreen(self, event=None):
        self.fullscreen = not self.fullscreen
        self.root.attributes("-fullscreen", self.fullscreen)

    def exit_fullscreen(self, event=None):
        self.fullscreen = False
        self.root.attributes("-fullscreen", False)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(
            text="Dibuixa una lletra i prem «Predir».", fg=self.text
        )
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()
        self.predictions_frame.pack_forget()

    # Predicció
    def preprocess(self):
        img_resized = self.image.resize((28, 28))
        img_array = np.array(img_resized)

        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        _, img_array = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        img_ready = np.expand_dims(img_array, axis=(0, -1))
        return img_ready

    def show_predictions(self, pred):
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()

        self.predictions_frame.pack(pady=10)

        sorted_indices = np.argsort(pred[0])[::-1]
        for idx in sorted_indices[:5]:
            percent = float(pred[0][idx]) * 100
            letter = chr(idx + 65)

            frame = tk.Frame(self.predictions_frame, bg=self.bg)
            frame.pack(fill="x", padx=60, pady=2)

            lbl = tk.Label(frame, text=f"{letter}: {percent:.1f}%", width=10,
                           bg=self.bg, fg=self.text, anchor="w", font=("Segoe UI", 11, "bold"))
            lbl.pack(side="left")

            bar = ttk.Progressbar(frame, length=300, value=percent, maximum=100)
            bar.pack(side="left", padx=8)
            self.pred_bars.append(frame)

    def predict(self):
        img_ready = self.preprocess()
        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))
        predicted_letter = chr(pred_class + 65)

        self.result_label.config(
            text=f"Lletra predita: {predicted_letter}", fg=self.accent
        )
        self.show_predictions(pred)


# Executar
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


MODEL 5 (NOMÉS DÍGITS):

In [2]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model #type: ignore


# Carrega del model (actualitza el path si cal)
model = load_model(r"C:\Users\joana\Documents\Projectes personals\TR\projecte\model5_cnn.keras")


class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconeixedor de Dígits")
        self.root.geometry("600x700")

        # Estat
        self.dark_mode = False
        self.fullscreen = False

        self.root.bind("<F11>", self.toggle_fullscreen)
        self.root.bind("<Escape>", self.exit_fullscreen)

        # Configura tema inicial
        self.set_theme()

        # Títol
        self.header = tk.Label(
            self.root, text="Reconeixedor de Dígits amb IA",
            font=("Segoe UI", 18, "bold"), bg=self.bg, fg=self.accent
        )
        self.header.pack(pady=(20, 10))

        # Canvas
        self.canvas_frame = tk.Frame(self.root, bg=self.card_bg, highlightbackground=self.border, highlightthickness=1)
        self.canvas_frame.pack(pady=15, ipadx=10, ipady=10)

        self.canvas = tk.Canvas(
            self.canvas_frame, width=300, height=300, bg=self.canvas_bg,
            highlightthickness=2, highlightbackground=self.border
        )
        self.canvas.pack(expand=True)

        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)

        # Botons
        self.btn_frame = tk.Frame(self.root, bg=self.bg)
        self.btn_frame.pack(pady=10)

        self.btn_predict = ttk.Button(self.btn_frame, text="Predir", command=self.predict)
        self.btn_clear = ttk.Button(self.btn_frame, text="Esborrar", command=self.clear)
        self.btn_theme = ttk.Button(self.btn_frame, text="Mode Fosc", command=self.toggle_theme)

        self.btn_predict.grid(row=0, column=0, padx=10)
        self.btn_clear.grid(row=0, column=1, padx=10)
        self.btn_theme.grid(row=0, column=2, padx=10)

        # Resultat
        self.result_label = tk.Label(
            self.root,
            text="Dibuixa un dígit i prem «Predir».",
            font=("Segoe UI", 14, "bold"),
            bg=self.bg, fg=self.text
        )
        self.result_label.pack(pady=10)

        # Contenidor prediccions
        self.predictions_frame = tk.Frame(self.root, bg=self.bg)
        self.predictions_frame.pack_forget()
        self.pred_bars = []

        # Binding ratolí
        self.canvas.bind("<B1-Motion>", self.paint)

    # ------------------ TEMES ------------------
    def set_theme(self):
        if self.dark_mode:
            self.bg = "#2b1e17"
            self.card_bg = "#3b2b23"
            self.canvas_bg = "#f2e9e1"
            self.text = "#f5ede3"
            self.accent = "#d8a657"
            self.border = "#5a4539"
            self.bar_color = "#d8a657"
        else:
            self.bg = "#f4f1ed"
            self.card_bg = "#ffffff"
            self.canvas_bg = "#ffffff"
            self.text = "#3f3a36"
            self.accent = "#b45309"
            self.border = "#d6d3ce"
            self.bar_color = "#b45309"

        self.root.configure(bg=self.bg)

        style = ttk.Style()
        style.theme_use("clam")
        style.configure(
            "TButton",
            font=("Segoe UI", 11, "bold"),
            padding=6,
            background=self.accent,
            foreground="white",
            borderwidth=0,
        )
        style.map("TButton", background=[("active", self.bar_color)])
        style.configure(
            "TProgressbar",
            troughcolor=self.card_bg,
            background=self.bar_color,
            bordercolor=self.bar_color,
        )

    def toggle_theme(self):
        self.dark_mode = not self.dark_mode
        self.set_theme()
        self.root.configure(bg=self.bg)
        self.header.configure(bg=self.bg, fg=self.accent)
        self.result_label.configure(bg=self.bg, fg=self.text)
        self.canvas.configure(bg=self.canvas_bg, highlightbackground=self.border)
        self.canvas_frame.configure(bg=self.card_bg, highlightbackground=self.border)
        self.predictions_frame.configure(bg=self.bg)
        self.btn_frame.configure(bg=self.bg)
        for frame in self.pred_bars:
            frame.configure(bg=self.bg)
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Label):
                    widget.configure(bg=self.bg, fg=self.text)

    # ------------------ FUNCIONS BÀSIQUES ------------------
    def toggle_fullscreen(self, event=None):
        self.fullscreen = not self.fullscreen
        self.root.attributes("-fullscreen", self.fullscreen)

    def exit_fullscreen(self, event=None):
        self.fullscreen = False
        self.root.attributes("-fullscreen", False)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (300, 300), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="Dibuixa un dígit i prem «Predir».", fg=self.text)
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()
        self.predictions_frame.pack_forget()

    # ------------------ PREDICCIÓ ------------------
    def preprocess(self):
        img_resized = self.image.resize((28, 28))
        img_array = np.array(img_resized)

        if np.mean(img_array) > 127:
            img_array = cv2.bitwise_not(img_array)

        _, img_array = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        img_ready = np.expand_dims(img_array, axis=(0, -1))
        return img_ready

    def show_predictions(self, pred):
        for w in self.pred_bars:
            w.destroy()
        self.pred_bars.clear()

        self.predictions_frame.pack(pady=10)

        sorted_indices = np.argsort(pred[0])[::-1]
        for idx in sorted_indices[:5]:
            percent = float(pred[0][idx]) * 100
            digit = str(idx)

            frame = tk.Frame(self.predictions_frame, bg=self.bg)
            frame.pack(fill="x", padx=60, pady=2)

            lbl = tk.Label(frame, text=f"{digit}: {percent:.1f}%", width=10,
                           bg=self.bg, fg=self.text, anchor="w", font=("Segoe UI", 11, "bold"))
            lbl.pack(side="left")

            bar = ttk.Progressbar(frame, length=300, value=percent, maximum=100)
            bar.pack(side="left", padx=8)
            self.pred_bars.append(frame)

    def predict(self):
        img_ready = self.preprocess()
        pred = model.predict(img_ready)
        pred_class = int(np.argmax(pred))
        predicted_digit = str(pred_class)

        self.result_label.config(text=f"Dígit predit: {predicted_digit}", fg=self.accent)
        self.show_predictions(pred)


# ------------------ EXECUCIÓ ------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
